In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import scipy.io as spio
import os
import re
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm

import matplotlib

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [38]:
# Create features data frame for a specific subject

from tkinter import Tk, filedialog
import glob

root = Tk()  # pointing root to Tk() to use it as Tk() in program.
root.withdraw()  # Hides small tkinter window.
root.attributes('-topmost', True)  # Opened windows will be active. above all windows despite selection.
path = filedialog.askdirectory()  # Returns opened path as str



In [39]:
# Choose a folder
dir = glob.glob(os.path.join(path, "*", ""), recursive=True)

In [40]:

def get_Features(dir, i):
    movs = glob.glob(os.path.join(dir[i] + "/*.avi"))
    vidcap = cv2.VideoCapture(movs[0])
    success, image = vidcap.read()
    print("Able to read movie?: " + str(success))
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    # load facial points
    dfh5_file = glob.glob(os.path.join(dir[i] + "/*30000.h5"))
    dfh5 = pd.read_hdf(dfh5_file[0])

    # Clac Center of mass
    row = dfh5.iloc[[2]]
    row = np.squeeze(row.to_numpy())
    nose = row[:2]
    ear = row[12:14]
    center_of_mass = np.mean([nose, ear], axis=0)

    # Subtract CoM
    dfh5.iloc[:, [0, 3, 6, 9, 12]] -= center_of_mass[0]
    dfh5.iloc[:, [1, 4, 7, 10, 13]] -= center_of_mass[1]

    # convert to np
    pos_df = dfh5.iloc[1:, [0, 1, 3, 4, 6, 7, 9, 10, 12, 13]]

    pos_df.to_csv(os.path.join(dir[i] + "out.csv"))
    pos = pos_df.to_numpy()

    # pupil
    dfh5_pupil_file = glob.glob(os.path.join(dir[i] + "/*70000.h5"))
    if dfh5_pupil_file:
        dfh5_pupil = pd.read_hdf(dfh5_pupil_file[0])
        dfh5_pupil.shape

        dist = lambda x, y: np.linalg.norm(x - y)

        pupil = dfh5_pupil.iloc[1:, [0, 1, 3, 4, 6, 7, 9, 10]]
        pupil = pupil.to_numpy()

        # i_up = pupil[:, 0:2] |  i_down = pupil[:, 2:4]  |  i_right = pupil[:, 4:6] |  i_left = pupil[:, 6:8]
        dist_hor = list(map(dist, pupil[:, 0:2], pupil[:, 4:6]))
        dist_ver = list(map(dist, pupil[:, 4:6], pupil[:, 6:8]))

        pos_n_pupil = np.c_[pos, dist_hor, dist_ver]


    else:
        print("--The mouse have no pupil file")
        pos_n_pupil = pos

    return pos_n_pupil, pos_df, fps


In [41]:
def getData(dir, before_q, after_q, add_prev_q=False):
    df = pd.DataFrame(
        columns=['Name', 'Task Level', 'Lick Rate [%]', 'True Positive', 'False Positive', 'Total Accuracy', 'Features',
                 'Labels'])

    for i in range(dir.__len__()):
        print("\nMouse " + str(i + 1))
        print(dir[i])
        print("------------------")

        file = os.path.normpath(dir[i]).split(os.path.sep)

        pos_n_pupil, pos_df, fps = get_Features(dir, i)
        if pos_n_pupil.shape[1] < 11:
            print("**SKIP**\n")
            continue
        # run matlab code if necessary ("Z:\Shared\Amichai\importfile.m")

        # get session data
        data = spio.loadmat(os.path.join(dir[i] + 'data.mat'))

        correction = pos_n_pupil.shape[0] / ((data['time'].shape[1] / data['Fs']) * fps)

        tSOS_new = data['tSOS'] * correction
        Lick_times_new = np.ravel(data['Lick_times'] * correction + (
                tSOS_new[0] - np.minimum(data['Go_times'][0, 0], data['NoGo_times'][0, 0])))

        stimuli = np.ravel(np.sort(np.c_[data['Go_times'], data['NoGo_times']]))
        stimuli_type = np.in1d(stimuli, data['Go_times'])

        if len(stimuli) != len(tSOS_new):
            print("--Can't label the cue type\n **SKIP** \n ")
            continue

        # movie interpolation
        r = pd.RangeIndex(0, int((data['time'].shape[1] / data['Fs']) * fps), 1)
        t = pos_df
        t = t.sort_index()
        new_idx = np.linspace(t.index[0], len(r), len(r))
        t = (t.reindex(new_idx, method='ffill', limit=1).iloc[1:].interpolate())

        pos_n_pupil = t.to_numpy()

        # segment to trails
        Lick_times = Lick_times_new * fps
        for idx, seg in enumerate(tSOS_new * fps):
            segment = pos_n_pupil[int(seg - before_q * fps):int(seg), :].reshape(
                [int(before_q * fps) * pos_n_pupil.shape[1]])
            lick_seg = np.any(
                np.ravel(np.where((Lick_times > seg) & (Lick_times < seg + after_q * fps))))

            if idx == 0:

                prv_lick = np.array([0])  # previous segment lick choice as a feature
                x = np.r_[segment, prv_lick]

                y = lick_seg


            else:
                y = np.row_stack((y, lick_seg))
                prv_lick = y[-2]  # previous lick choice
                segment = np.r_[segment, prv_lick]
                x = np.row_stack((x, segment))

        lick_bias = y.sum() / y.shape[0]

        if lick_bias < 0.20 or lick_bias > 0.80:
            print("**Biased Licker**")
            continue
        level = np.full_like(y, re.findall("[he]", file[-1]))
        print(level[0])
        name = np.full_like(y, i)
        print(name[0])
        # concat this mouse data to the others
        # create empty arrays
        if i == 0:
            X = x
            y_tot = y
            stimuli_tot = stimuli_type
            level_tot = level
            name_tot = name



        else:
            X = np.concatenate((X, x), axis=0)

            y_tot = np.concatenate((y_tot, y), axis=0)
            stimuli_tot = np.concatenate((stimuli_tot, stimuli_type), axis=0)
            level_tot = np.concatenate((level_tot, level), axis=0)
            name_tot = np.concatenate((name_tot, name), axis=0)

        TP = len(data['Reward_times'][0]) / len(data['Go_times'][0])
        try: FP = len(data['Punishment_times'][0]) / len(data['NoGo_times'][0])
        except: FP = 0
        TN = 1 - FP
        FN = 1 - TP
        Acc = (TP + TN) / (TP + TN + FP + FN)

        mouse = pd.DataFrame({
            'Name': re.findall("A.._", file[-1]),
            'Task Level': re.findall("[he]", file[-1]),
            'Lick Rate [%]': lick_bias * 100,
            'True Positive': TP,
            'False Positive': FP,
            'Total Accuracy': Acc,
            'Features': [x],
            'Labels': [y]}, )
        df = pd.concat([df, mouse], ignore_index=True)
    return X, np.ravel(y_tot), np.ravel(stimuli_tot), df


In [43]:


print("missing pupil:")
for i in range(dir.__len__()):
    file_name = glob.glob(os.path.join(dir[i] + "/*70000.h5"))
    if file_name == []:
        print(dir[i])

print("missing pos:")
for i in range(dir.__len__()):
    file_name = glob.glob(os.path.join(dir[i] + "/*30000.h5"))
    if file_name == []:
        print(dir[i])


missing pupil:
missing pos:


In [44]:
# Run again?
run_again = False
##########################################################################

os.chdir(path)
if run_again:
    # Get the data and save it
    X, y, stimuli_type, mice_data = getData(dir=dir, before_q=0.5, after_q=2)

    mice_data.to_pickle('./Mice_Data.pkl')
    np.savez('./Features_n_label', X=X, y=y, stimuli_type=stimuli_type)

else:
    # try to read data, if not exist run the whole session
    try:
        mice_data = pd.read_pickle('Mice_Data.pkl')
        data = np.load('Features_n_label.npz')
        X = data['X']
        y = data['y']
        stimuli_type = data['stimuli_type']

    except:
        X, y, stimuli_type, mice_data = getData(dir=dir, before_q=0.5, after_q=2)
        mice_data.to_pickle('./Mice_Data.pkl')
        np.savez('./Features_n_label', X=X, y=y, stimuli_type=stimuli_type)


In [45]:
add_prev_q = True
add_q = False
add_answer = False


# Add the cue type
if add_prev_q:
    x = X
else:
    x = X[:, :-1]

if add_q:
    x = np.c_[x, stimuli_type]

# Add the answer
if add_answer:
    x = np.c_[x, y]


In [46]:
K = 1000
acc = np.zeros([K, 2])

for k in range(K):
    print(str(k) + "/" + str(K), end="\r")
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Training and Making Predictions
    # classifier = GaussianNB()
    classifier = svm.SVC(kernel='rbf')
    # classifier = RandomForestClassifier(max_depth=2, random_state=0)

    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    acc[k, 0] = accuracy_score(y_test, y_pred)
    train_pred = classifier.predict(X_train)
    acc[k, 1] = accuracy_score(y_train, train_pred)

# Evaluating the Performance
print('Train accuracy: ' + str(round(np.mean(acc[:, 1]), 2)) + ' || STD: ' + str(round(np.std(acc[:, 1]), 3)))
print('Test accuracy: ' + str(round(np.mean(acc[:, 0]), 2)) + ' || STD: ' + str(round(np.std(acc[:, 0]), 2)))

lick_bias = y.sum() / y.shape[0] * 100

print(
    "---------------------------------------\nLick Events: ~" + str(round(lick_bias)) + "% || No-lick Events: ~" + str(
        round(100 - lick_bias)) + "%")

cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='Greys')


Train accuracy: 0.64 || STD: 0.006
Test accuracy: 0.55 || STD: 0.02
---------------------------------------
Lick Events: ~51% || No-lick Events: ~49%
